## Music Generation using LSTMs



In [1]:
import os
import json 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import *
from music21 import *
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
BATCH_SIZE = 16
SEQ_LENGTH = 128

ABC Notation

ABC notation of Music
There are two parts in ABC-notation.

Part-1 represents meta data. Lines in the Part-1 of the tune notation, beginning with a letter followed by a colon, indicate various aspects of the tune such as the index, when there are more than one tune in a file (X:), the title (T:), the time signature (M:), the default note length (L:), the type of tune (R:) and the key (K:).

Part-2 represents the tune, which is a sequence of characters where each character represents some musical note.

In [3]:
# Function to remove Part-1 of  and append a start token i.e 'Z' for each tune in the dataset

def preprocess(data):
  list1=list(data)
  list2=['\n','\n','\n']
  ignore=['X','T','M','S','K','P']
  i=0
  #to remove Part1:
  while(i<len(list1)):
    if(((list1[i] in ignore) and (list1[i+1]==":"))or list1[i]=='%' ):
      del list2[-1]
      while(list1[i]!='\n'):
        i=i+1
    list2.append(list1[i])
    i=i+1
  i=0
  #to append 'Z'(start token)
  preprocess_data=[]
  while(i<len(list2)):
    if(list2[i]=='\n'and list2[i+1]=='\n' and list2[i+2]=='\n'):
      preprocess_data.append('Z')
      i=i+3
    else:
      preprocess_data.append(list2[i])
      i=i+1
  return preprocess_data


In [4]:
# Function to create char_to_index and index_to_char dictionaries so as to map each character to an index and vice versa.
# Returns all_characters_as_indices i.e an array containing all characters of the dataset replaced with their corresponding indices as per the vocabulary.
# Also returns num_unique_chars i.e an integer equal to number of unique characters in the data.

def read_data(preprocess_data):
  char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(preprocess_data))))}

    
  with open(charIndex_json, mode = "w") as f:
        json.dump(char_to_index, f)
        
  index_to_char = {i: ch for (ch, i) in char_to_index.items()}
  num_unique_chars = len(char_to_index)
  all_characters_as_indices = np.asarray([char_to_index[c] for c in preprocess_data], dtype = np.int32)
  return all_characters_as_indices,num_unique_chars



In [5]:
# Function which returns X and Y which will be used as input and target output for training the model.

def input_output(all_chars_as_indices,num_unique_chars):
    total_length = all_chars_as_indices.shape[0]
    num_examples=int(total_length/SEQ_LENGTH)
    X=np.zeros((num_examples,SEQ_LENGTH))
    Y=np.zeros((num_examples,SEQ_LENGTH,num_unique_chars))
    for i in range(num_examples):
      for j in range(SEQ_LENGTH):
        X[i,j]=all_chars_as_indices[i*SEQ_LENGTH+j]
        Y[i,j,all_chars_as_indices[i*SEQ_LENGTH+j+1]]=1
    return X,Y

In [6]:
# Function to build the training model

def build_model( seq_length, num_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, input_shape = (seq_length,))) 
    
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    
    model.add(TimeDistributed(Dense(num_unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [7]:
# Image(filename='./drive/My Drive/build.png') #image of training model

stateful:

 If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.

In [8]:
#Function which builds model for generating music sequences.

def make_model(num_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
# stateful: If True, the last state for each sample at index i in a batch will be used 
# as initial state for the sample of index i in the following batch.
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256,return_sequences=True, stateful = True)) 
    model.add(Dropout(0.2))
    
    model.add((Dense(num_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [9]:
# Image(filename='./drive/My Drive/generate.png') #image showing music generation model.

In [10]:
# Function which generates music sequences of length=gen_seq_length.
def generate_sequence(gen_seq_length):
    with open(charIndex_json) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    num_unique_chars = len(index_to_char)
    
    model = make_model(num_unique_chars)
    model.load_weights("./weights/weights.79.hdf5")
     
    sequence_index = [char_to_index['Z']]

    for _ in range(gen_seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(num_unique_chars), size = 1, p = predicted_probs)
        
        
        sequence_index.append(sample[0])
    
        
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    seq='M:6/8\n'+str(seq)
    return seq

In [11]:
# function to create a midi file given a music sequence in abc notation.
def convert_to_midi(abc):
    c = converter.subConverters.ConverterABC()
    c.registerOutputExtensions = ("midi")
    c.parseData(abc)
    s = c.stream
    s.write('midi', fp='demos1.mid')

In [12]:
import sys

file = open(data_file, mode = 'r')
data = file.read()
file.close()
preprocess_data=preprocess(data)
all_characters_as_indices,num_unique_chars=read_data(preprocess_data)
X,Y=input_output(all_characters_as_indices,num_unique_chars)
print("length of preprocess_data-{}".format(len(preprocess_data)))
print("vocab_size={}".format(num_unique_chars))
print("all_characters={}".format(all_characters_as_indices))
print("length of all_characters-{}".format(len(all_characters_as_indices)))
print("shape of X={}".format(X.shape))
print("shape of Y={}".format(Y.shape))

length of preprocess_data-116963
vocab_size=59
all_characters=[33 44 57 ... 15 20 57]
length of all_characters-116963
shape of X=(913, 128)
shape of Y=(913, 128, 59)


In [13]:
model=build_model(SEQ_LENGTH,num_unique_chars)
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 512)          30208     
_________________________________________________________________
lstm (LSTM)                  (None, 128, 256)          787456    
_________________________________________________________________
dropout (Dropout)            (None, 128, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128, 256)          525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128, 256)          525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 256)          0

In [14]:
checkpoint=ModelCheckpoint(filepath='./weights/weights.{epoch:02d}.hdf5',monitor='loss',save_best_only=True,save_weights_only=True,period=1)
model.fit(X,Y,batch_size=BATCH_SIZE,epochs=80,callbacks=[checkpoint])


Epoch 1/80
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
58/58 [==============================] - 3s 47ms/step - loss: 3.1574 - accuracy: 0.1778
Epoch 2/80
58/58 [==============================] - 3s 47ms/step - loss: 2.8242 - accuracy: 0.2370
Epoch 3/80
58/58 [==============================] - 3s 50ms/step - loss: 2.2861 - accuracy: 0.3629
Epoch 4/80
58/58 [==============================] - 3s 47ms/step - loss: 1.9217 - accuracy:

58/58 [==============================] - 4s 62ms/step - loss: 0.3211 - accuracy: 0.8909
Epoch 73/80
58/58 [==============================] - 4s 63ms/step - loss: 0.3252 - accuracy: 0.8906
Epoch 74/80
58/58 [==============================] - 4s 67ms/step - loss: 0.3199 - accuracy: 0.8915
Epoch 75/80
58/58 [==============================] - 4s 65ms/step - loss: 0.3206 - accuracy: 0.8915
Epoch 76/80
58/58 [==============================] - 4s 63ms/step - loss: 0.2982 - accuracy: 0.8998
Epoch 77/80
58/58 [==============================] - 4s 63ms/step - loss: 0.2939 - accuracy: 0.8999
Epoch 78/80
58/58 [==============================] - 4s 63ms/step - loss: 0.2857 - accuracy: 0.9040
Epoch 79/80
58/58 [==============================] - 4s 61ms/step - loss: 0.2891 - accuracy: 0.9017
Epoch 80/80
58/58 [==============================] - 4s 62ms/step - loss: 0.2855 - accuracy: 0.9029


In [ ]:
music = generate_sequence(192)
print("\nMUSIC SEQUENCE GENERATED: \n{}".format(music))
convert_to_midi(music)


We have finished training and generating music sequences.

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [16]:
import tensorflow as tf